In [1]:
import pandas as pd
import numpy as np

from pycel import ExcelCompiler
import time

In [2]:
def Convert_Column_Times_To_Seconds(time_column):
    return_list = []
    for decimal_time in time_column:
        return_list.append(decimal_time * 24 * 3600)

    return return_list

In [3]:
def Convert_Column_Times_to_Format(time_column):
    return_list = []
    for timestamp in time_column:
        return_list.append(time.strftime('%H:%M:%S', time.gmtime(timestamp)))

    return return_list    

In [4]:
# If True, the Excel model is evaluated again, and exported to CSV as cache. If False, cached CSV files are used.
generate_model_cache = True

aircraft_types = ["A350", "A380"] #, "A350", "A380", "B788", "B77W", "B748"]

In [5]:
# Evaluate model and store results in dataframe, and export CSV file as cache

if(generate_model_cache == True):

    flight_df_dict = {}

    # Column names of emissions Dataframe
    column_names = ["Distance [nm]", "Cruise flight level [100 ft]", "Duration [Excel]", "Fuel burn [kg]", "CO2 [kg]", "NOx [kg]", "SOx [kg]", "H2O [kg]", "CO [kg]", "HC [kg]", "PM non volatile [kg]", "PM volatile [kg]", "PM total [kg]"]

    for aircraft_type in aircraft_types:
        model_path = "model/EEAEmissionsCalculator_{}.xlsm".format(aircraft_type)
        model = ExcelCompiler(filename=model_path)

        # print("Previous aircraft setting: {}".format(model.evaluate('Emissions calculator!D17')))

        # # Set settings in Excel model to aircraft type
        # model.set_value('Emissions calculator!D17', aircraft_type)

        print("Aircraft setting: {}".format(model.evaluate('Emissions calculator!D17')))

        # Evaluate model for this aircraft type and store results in dictionary
        evaluation_results = model.evaluate('Emissions calculator!D33:P42')
        evaluation_results_df = pd.DataFrame.from_dict(evaluation_results)
        evaluation_results_df.columns = column_names

        # Convert times to seconds
        evaluation_results_df.insert(loc=0, column='Duration [s]', value=Convert_Column_Times_To_Seconds(evaluation_results_df["Duration [Excel]"]))

        # Adding rows of zeros to set initial condition for further calculations
        evaluation_results_df.loc[-1] = ['0','0','0','0','0','0','0','0','0','0','0','0','0','0']
        evaluation_results_df.index = evaluation_results_df.index + 1 # shifting index
        evaluation_results_df = evaluation_results_df.sort_index()  # sorting by index
        evaluation_results_df = evaluation_results_df.apply(pd.to_numeric)

        # Excel time format is no longer needed
        evaluation_results_df = evaluation_results_df.drop(columns='Duration [Excel]')

        # Store CSV for cache purposes
        evaluation_results_df.to_csv("model/{}.csv".format(aircraft_type))

        flight_df_dict[aircraft_type] = evaluation_results_df
    


C:\Users\marko\anaconda3\envs\LCAero\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
C:\Users\marko\anaconda3\envs\LCAero\lib\site-packages\openpyxl\reader\drawings.py:29: UserWarning: DrawingML support is incomplete and limited to charts and images only. Shapes and drawings will be lost.
  warn("DrawingML support is incomplete and limited to charts and images only. Shapes and drawings will be lost.")


Aircraft setting: A350
Aircraft setting: A380


In [ ]:
def move_column_inplace(df, col, pos):
    col = df.pop(col)
    df.insert(pos, col.name, col)

    return df

In [ ]:
def Convert_Excel_Times(decimal_time):
    hours = decimal_time * 24
    minutes = 60 * hours
    seconds = 60 * minutes

    time_return = time.strftime('%H:%M:%S', time.gmtime(seconds))
    return time_return

In [ ]:
def Convert_Column_Times(time_column):
    return_list = []
    for decimal_time in time_column:
        return_list.append(Convert_Excel_Times(decimal_time))

    return return_list

In [ ]:
a320_df = pd.read_csv("model/a320.csv", index_col=0)

# Remove first rows
a320_df = a320_df.iloc[2:]

a320_df = a320_df.apply(pd.to_numeric)

# Convert times to second
a320_df.insert(loc=0, column='Duration [s]', value=Convert_Column_Times_To_Seconds(a320_df["Duration [Excel]"]))

# Drop other time columns
a320_df = a320_df.drop(columns='Duration [hh:mm:ss]')
a320_df = a320_df.drop(columns='Duration [Excel]')

#a320_df = move_column_inplace(a320_df, "Duration [Excel]", 0)

In [ ]:
a320_df.loc[-1] = ['0','0','0','0','0','0','0','0','0','0','0','0','0']  # adding a row
a320_df.index = a320_df.index + 1  # shifting index
a320_df = a320_df.sort_index()  # sorting by index
a320_df = a320_df.apply(pd.to_numeric)


In [ ]:
def Fetch_Rows_For_Interpolation(flight_df, time_given):
    """Fetches the two rows in the data next to the given time (closest lower than given time and closest higher than given time), used for linear interpolation"""
    
    df_mask = flight_df['Duration [s]'] <= time_given

    # Get last row where time is lower than given time
    flight_df_lower_bound = flight_df[df_mask].iloc[-1:]
    
    index_lower_bound = flight_df_lower_bound.iloc[0:].index
    
    # Look up position of index in non-filtered dataframe
    loc_lower_bound = flight_df.index.get_loc(index_lower_bound[0])

    # Upper bound is one position (row) higher in Dataframe
    loc_upper_bound = loc_lower_bound + 1

    flight_df_upper_bound = flight_df.iloc[[loc_upper_bound]]

    df_list = [flight_df_lower_bound, flight_df_upper_bound]
    flight_df_interval = pd.concat(df_list)

    return flight_df_interval

In [ ]:
def Interpolate_Emissions(flight_df, given_times):
    """Receives list of time points for which emissions are to be interpolated
    :return: Dataframe with emissions for each time point
    """

    df_list = []

    for given_time in given_times:

        # Get data higher and lower than given time
        flight_df_interval = Fetch_Rows_For_Interpolation(flight_df, given_time)

        # Init dictionary
        emissions_at_time = {}

        x = given_time

        # Iterate through columns to interpolate each of them individually
        for (column_name, column_data) in flight_df_interval.iteritems():

            # Get x values (times) as interpolation boundaries on x-axis
            if(column_name == "Duration [s]"):
                x1 = flight_df_interval["Duration [s]"].iloc[0]
                x2 = flight_df_interval["Duration [s]"].iloc[1]

            y1 = column_data.iloc[0]
            y2 = column_data.iloc[1]

            # Linear interpolation
            y = y1 + (x - x1)*((y2-y1)/(x2-x1))

            # Store interpolated column value to dictionary
            emissions_at_time[column_name] = [y]

        emissions_at_time_df = pd.DataFrame.from_dict(emissions_at_time)
        emissions_at_time_df.insert(loc=1, column='Duration [hh:mm:ss]', value=Convert_Column_Times_to_Format(emissions_at_time_df["Duration [s]"]))

        df_list.append(emissions_at_time_df)
    
    emissions_at_times_df = pd.concat(df_list)

    return emissions_at_times_df


In [ ]:
desired_times = [100, 2000, 3000]

emissions_at_times_output_df = Interpolate_Emissions(a320_df, desired_times)

In [ ]:
emissions_at_times_output_df

# Notes

## EEA Model
Manipulating the EEA Aviation Emissions Calculator via the Python library PyCel is slow. Reading the "Climb, cruise, descent" stage length (NM) information from the Excel model for a flight with an Airbus A320 takes about 3.5 minutes.

The primary goal of this project is not the development of a Python implementation of the EEA Aviation Emissions Calculator. Therefore, we do not focus on increasing the excecution speed of the Excel model in Python. Instead, we cache the information. The model is executed in Excel with standard settings for each aircraft that should be covered by LCAero. The results (emissions) are cached in a CSV file that can be imported by a Python script without analyzing the spreadsheet again.

- Add simple explaination of EEA model to report, explain different flight phases, etc.
- Define LTO and CCD stage length

## Other life data

Flight cycles is very hard to determine as there are so many variables. Every airline has its own rules on this. There is not an x amount of maximum flying cycles a seat or galley is designed for. 
It also varies per product (seat, galley, lavatory etc) and sort of aircraft and mission (low cost vs. full service, long haul vs short/medium range).

A low cost airline has much more wear and tear on the seats, carpet (sometimes 8-10 short flight cycles a day) but these aircraft have less wear and tear on the galley as there is much less on board service and catering available and also less wear and tear on the lavatory since people use the bathroom less on short flights. 

So for example a Boeing 737 from KLM (full service airline with maybe max 4 flights a day with catering) will have a completely different interior replacement schedule than a Boeing 737 from Ryanair (low cost with maybe 8 to 10 flight a day and no catering) . 

Also there is a difference in long haul and short range. The latter has much more flying cycles and therefore the seats and carpets wear much faster as there are more people moving through the isle and using the overhead bins during one day. 

Seats and lavatories are usually replaced when the aircraft is checked during a C- check . This can be a certain time frame (e.g. 2 years) or 2000-3000 flight cycles.
Galleys are replaced during D-checks (every 10 years, or 10000-15000 cycles) but not per se during every D check. Depends on how long the airline is planning to continue flying with the A/C

So there is not a definitive answer on the number of flight cycles. 

Criteria like environmental performance is now being introduced within the airline industry. But it is very new for most. For some airlines ((Northern) Europe) this is  a factor that they take into account when replacing equipment. This applies especially to airlines that are (partially) owned by the local government as they usually set is as a requirement for new purchases. For others it is not important at all (Africa , Central Asia and Middle East).

I don’t have access to live data from A/C at the moment, same goes for emissions calculator.  Think this is also company confidential info. 
